# Analyse der Internetnutzung und Schülerleistung

In dieser Analyse untersuchen wir den Zusammenhang zwischen der Internetnutzung und der schulischen Leistung von Schülern. Wir betrachten, wie die Nutzung des Internets das Lernverhalten, die Motivation und die Ergebnisse der Schüler beeinflusst.

Durch die Analyse von Daten zur Internetnutzung und schulischen Leistung können wir wichtige Erkenntnisse gewinnen, die uns helfen, Bildungsstrategien zu verbessern und den Schülern dabei zu helfen, ihr volles Potenzial auszuschöpfen.

Wir werden verschiedene Variablen betrachten, wie z.B. die Zeit, die Schüler online verbringen, die Art der Internetnutzung (z.B. für schulische Zwecke oder Freizeitaktivitäten) und die Auswirkungen auf ihre schulischen Leistungen. Darüber hinaus werden wir auch demografische und sozioökonomische Faktoren berücksichtigen, um ein umfassendes Bild zu erhalten.

Zu Beginn werden die Daten geladen und eine explorative Datenanalyse durchgeführt, um die Verteilung der Variablen zu untersuchen und mögliche Zusammenhänge zu identifizieren. Anschließend werden wir verschiedene statistische Methoden anwenden, um den Zusammenhang zwischen Internetnutzung und Schülerleistung zu untersuchen.


TODO score design für bildungs maßnahmen ergriffen von den Schülern selbst

BildungsScore = Internetnutzung * (inf - com - soc) + Bildungsfreizeit + Analoge Inforamtionen

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import statsmodels.api as sm
from statsmodels.formula.api import ols

path_prefix = 'Anabell/Daten/'
path_prefix2 = 'Daten/'

# Load the data
pisa_math = pd.read_csv(path_prefix+'pisaMathe.csv', index_col='Jahr', parse_dates=['Jahr'])
pisa_nature = pd.read_csv(path_prefix+'pisaNatur.csv', index_col='Jahr', parse_dates=['Jahr'])
pisa_reading = pd.read_csv(path_prefix+'pisaLese.csv', index_col='Jahr', parse_dates=['Jahr'])

internet_usage = pd.read_csv(path_prefix+'internetNutzung.csv', index_col='Jahr', parse_dates=['Jahr']).xs('InternetMinuten', drop_level=False, axis=1)
print_usage = pd.read_csv(path_prefix+'printNutzung.csv', index_col='Jahr', parse_dates=['Jahr'])
politics = pd.read_csv(path_prefix+'politischesInteresse.csv', index_col='Jahr', parse_dates=['Jahr'])
music_school_attendance = pd.read_csv(path_prefix+'schulerMusikschulen.csv', index_col='Jahr', parse_dates=['Jahr'])
number_of_students = pd.read_csv(path_prefix+'schulerAnzahl.csv', index_col='Jahr', parse_dates=['Jahr'])
#music_schools = pd.read_csv(path_prefix+'musikSchulenAnzahl.csv', index_col='Jahr', parse_dates=['Jahr'])
inet_usage_cat = pd.read_csv(path_prefix+'JIM.csv', index_col=['Aktivität','Kategorie']).groupby('Kategorie').sum()
inet_usage_cat = inet_usage_cat.T.sort_index()
inet_usage_cat.index.name = 'Jahr'
# extend index to 2000
new_idx = [str(i) for i in range(2000, 2024)]
# set 0 to na
inet_usage_cat = inet_usage_cat.replace(0, np.nan)
inet_usage_cat = inet_usage_cat.reindex(new_idx).bfill()
inet_usage_cat.index = pd.to_datetime(inet_usage_cat.index)

# idea: the upraising pisa scores where interupted by the introduction of social media
# TODO: anzahl schüler im verältnis zu schülerzahl insgesamt
# TODO: Delta score vllt besser um impact zu zeigen


In [3]:
# Merge the data
data_raw = pd.concat([pisa_math, pisa_nature, pisa_reading, print_usage, politics, music_school_attendance, internet_usage, inet_usage_cat], axis=1)
data = data_raw.sort_index().ffill().bfill()
data = data.loc['2000':]
#data = data / data.iloc[0,:]
data['score'] = (data['Mathematik'] + data['Naturwissenschaften'] + data['Lesekompetenz']) / 3 
data['score_diff'] = data['score'].diff()
data['score_diff'] = data['score_diff'].replace(0, np.nan).bfill()

# education score
data['education_score'] = data['InternetMinuten'] / 1000 * (data['inf'] - data['soc'] - data['com']) + data['AnzahlSchuler'] / 900000 + (data['Tageszeitung'] + data['Zeitschriften_Magazine'] + data['Bücher']) / 3

In [4]:
data

,Mathematik,Naturwissenschaften,Lesekompetenz,Bücher,Zeitschriften_Magazine,Tageszeitung,PolitikInteresse(prozent),AnzahlSchuler,InternetMinuten,com,inf,soc,score,score_diff,education_score
Jahr,,,,,,,,,,,,,,,
2000-01-01,490.0,487.0,484.0,41.0,33.0,48.0,30.0,890079.0,99.0,110.0,22.0,132.0,487.000000,11.666667,19.875643
2001-01-01,490.0,487.0,484.0,41.0,33.0,48.0,30.0,890079.0,99.0,110.0,22.0,132.0,487.000000,11.666667,19.875643
2002-01-01,490.0,487.0,484.0,41.0,33.0,48.0,30.0,890079.0,99.0,110.0,22.0,132.0,487.000000,11.666667,19.875643
2003-01-01,503.0,502.0,491.0,41.0,33.0,48.0,30.0,859903.0,99.0,110.0,22.0,132.0,498.666667,11.666667,19.842114
2004-01-01,503.0,502.0,491.0,41.0,33.0,48.0,30.0,888347.0,99.0,110.0,22.0,132.0,498.666667,6.333333,19.873719
2005-01-01,503.0,502.0,491.0,40.0,32.0,44.0,30.0,893538.0,99.0,110.0,22.0,132.0,498.666667,6.333333,17.879487
2006-01-01,504.0,516.0,495.0,40.0,30.0,47.0,35.0,903261.0,99.0,110.0,22.0,132.0,505.000000,6.333333,18.223623
2007-01-01,504.0,516.0,495.0,37.0,31.0,48.0,35.0,901091.0,106.0,110.0,22.0,132.0,505.000000,5.000000,16.347879
2008-01-01,504.0,516.0,495.0,40.0,29.0,43.0,35.0,930007.0,117.0,110.0,22.0,132.0,505.000000,5.000000,12.626674


In [6]:
px.imshow(data_raw.corr()) 

In [9]:
# plot pisa
plt_data = data[['Mathematik', 'Naturwissenschaften', 'Lesekompetenz', 'score']] - 500
plt_data = pd.concat([plt_data, data[['score_diff', 'education_score']]], axis=1)
fig = px.line(plt_data, y=['Mathematik', 'Naturwissenschaften', 'Lesekompetenz', 'score_diff', 'education_score', 'score'])
fig.update_layout(title='PISA Ergebnisse 2000-2024')
fig.update_yaxes(title='Punkte')
fig.show()

In [57]:
# plot activities
fig = px.line(data, y=['InternetMinuten',
       'Bücher', 'Zeitschriften_Magazine', 'Tageszeitung',
       'PolitikInteresse(prozent)', 'AnzahlSchuler'])
fig.update_layout(title='Aktivitäten 2000-2024')
fig.update_yaxes(title='Werte')
fig.show()

In [58]:
# plot com inf soc and vid
fig = px.line(data, y=['com', 'inf', 'soc'])
fig.update_layout(title='Internetnutzung 2000-2024')
fig.update_yaxes(title='Werte')
fig.show()

In [7]:
# fit a linear model with formula
formula = 'score_diff ~ education_score'
model = ols(formula, data).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             score_diff   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     229.1
Date:                Wed, 03 Jul 2024   Prob (F-statistic):           9.03e-13
Time:                        12:03:15   Log-Likelihood:                -57.788
No. Observations:                  23   AIC:                             119.6
Df Residuals:                      21   BIC:                             121.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.3504      0.656      0.534      0.599      -1.015       1.716
education_score     0.4175      0.028     15.137      0.000       0.360       0.475
==============================================================================
Omnibus:                        4.288   Durbin-Watson:                   1.703
Prob(Omnibus):                  0.117   Jarque-Bera (JB):                2.652
Skew:                          -0.803   Prob(JB):                        0.266
Kurtosis:                       3.430   Cond. No.                         24.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
# fit a linear model
X = data[['InternetMinuten', 'Bücher', 'Zeitschriften_Magazine', 'Tageszeitung', 'PolitikInteresse(prozent)', 'AnzahlSchuler']]
X = sm.add_constant(X)
y = data['score']

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.540
Method:                 Least Squares   F-statistic:                     5.498
Date:                Mon, 01 Jul 2024   Prob (F-statistic):            0.00252
Time:                        15:59:21   Log-Likelihood:                 71.639
No. Observations:                  24   AIC:                            -129.3
Df Residuals:                      17   BIC:                            -121.0
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         0.7467      0.149      5.018      0.000       0.433       1.061
InternetMinuten               0.0451      0.024      1.845      0.082      -0.006       0.097
Bücher                        0.0222      0.071      0.314      0.757      -0.127       0.171
Zeitschriften_Magazine       -0.0570      0.109     -0.521      0.609      -0.287       0.174
Tageszeitung                  0.1791      0.069      2.579      0.020       0.033       0.326
PolitikInteresse(prozent)     0.0886      0.063      1.417      0.175      -0.043       0.221
AnzahlSchuler                -0.0034      0.045     -0.074      0.942      -0.099       0.092
==============================================================================
Omnibus:                        0.550   Durbin-Watson:                   1.790
Prob(Omnibus):                  0.760   Jarque-Bera (JB):                0.627
Skew:                           0.144   Prob(JB):                        0.731
Kurtosis:                       2.262   Cond. No.                         196.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""